# Trade Bot Analysis

## Background 

Ripple's XRP is a top 5 market cap crypto currency traded on multiple markets. The price of XRP is highly volatile and commonly fluctuates multiple percent per day with the occasional bull/bear runs. 

One of the more popular exchanges for trading XRP is Bitstamp. Bitstamp has a open [api](https://www.bitstamp.net/api/) which allows the user to extract real time market data and information connected to the users account. This allows the user create trades through the api, giving the possibilty to set up trading automation in form of a bot which implements a trading strategy.  

# Trading Strategy

Using the daily fluctuation in price a naive strategy could be:

<ol>
  <li>Buy at a price X</li>
  <li>Sell at price Y, where price why is a big enough increase in price to gain some profit while covering fee</li>
  <li>Repeat the process when price is X using the intial value or inluding profits to purchase</li>
</ol>

### Problems
The price might go up enough to sell, furthermore the price might not go down again after selling. 

## Mathematical  Formulation and Analysis

#### Nomenclature
$P$: Profit \[\$\] <br>
$V_0$: Value of initial investment \[\$\]<br>
$i$: Interest [%] <br>
$f$: Trading Fee  [%] <br>
$n$: Number of succesful cycles <br>
$c$: Increase value [%] <br>

#### Profits Excluding Profits
The profit per succesful cycle when reinvesting **excluding** profits is given by,

$$P = V_0(1+i) - V_0(1+i)f - V_0f - V_0 = V_0((1+i) - (1+i)f - f - 1)$$
$$c = (1+i) - (1+i)f - f$$
$$P = V_0(c-1)$$

The potential total proft is given by,

$$ P_{tot} = \sum_{i=0}^n P_i = nP$$

#### Profits Including Profits
The profit per succesful cycle when reinvesting **including** profits is given by,

$$V_0 = V_0$$
$$V_1 = V_0c$$
$$V_2 = V_1c=V_0c²$$
$$V_3 = V_2c=V_0c³$$
$$\vdots$$
$$V_n = V_0c^n$$.
$$P_n = V_n - V_0 =  V_0c^n - V_0 = V_0 (c^n - 1)$$

#### Required Interest Rate
The minimum interest required to break $P=0$ even is, 

$$c = (1+i) - (1+i)f - f) = 1  \rightarrow i = \frac{f+1}{(1-f)}$$

The fee on bitstamp is 0.05% which gives

$$P(0.005) = 0.010050251256281229 \rightarrow 1.0050251256281229\% $$

as the break even interest rate. The change in profit based on changes in interest can be described by the derivative,

$$ \frac{dP}{di} = V_0(1-f) $$

the increase in profit based on changes in interest is constant. 

### Visualisation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go

# Constants
initial_value = 1000
fee = 0.005
interest = 0.015
value_increase = (1 + interest) - (1 + interest) * fee - fee
n = 100

succesful_cycles = np.arange(500)
profit_reinvesting_excluding_profit = [n * (value_increase - 1) * 100 for n in succesful_cycles]
profit_reinvesting_including_profit = [(value_increase**n - 1) * 100 for n in succesful_cycles]



In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=succesful_cycles, y=profit_reinvesting_excluding_profit,
                    mode='lines',
                    name='Excluding Profit'))
fig.add_trace(go.Scatter(x=succesful_cycles, y=profit_reinvesting_including_profit,
                    mode='lines',
                    name='Including Profit'))

# Edit the layout
fig.update_layout(title=f'Profit Per Succesful cycle with interest={interest*100}%, fee={fee*100}%',
                   xaxis_title='Succesfull Cycle',
                   yaxis_title='Profit [%]')
fig.show()

## Conclusion 
The interest should be around 1.5% and including profits gives a noticable increase in profits after around 100 successfull cycles.

## Algorithmic Implementation

<ol>
  <li>Buy in at previous day low price (assuming there is not bull/bear run)</li>
  <li>Check ask prices on a t sec interval</li>
  <li>If ask price is the required percent increase then sell</li>
  <li>Check ask prices on a t sec interval</li>
  <li>If ask price is = original buy price then buy </li>
  <li>Continue and repeat</li>
</ol>
